In [1]:
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import QuadraticProgramToQubo
from qiskit.optimization.algorithms.optimization_algorithm import OptimizationResultStatus
import numpy as np
from helpers import *
DIR = 'TEST_DATA/'
getPath = lambda fn : DIR+fn

In [2]:
qps = createModelsFromDir(DIR)
# solve qps with Minimum Eigen Optimizer
quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'))
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=quantum_instance))

In [3]:
for qp_name in sorted(qps.keys(), key = lambda name : int(name[name.rfind('_')+1:])) :
    print(qp_name)
    res = qaoa.solve(qps[qp_name])
    if res.status == OptimizationResultStatus.SUCCESS:
        print("success")
        cres  = qps[qp_name].model.solve()
        if res.fval != cres.get_objective_value():
            print("\n optimal value QAOA   "+str(res.fval) + "  ,  cplex:"+ str(cres.get_objective_value()))

test_2
success
* No objective to optimize - searching for a feasible solution
test_3
success

 optimal value QAOA   0.0  ,  cplex:-1.0
test_4


constraint A0_leb1 is infeasible due to substitution


success

 optimal value QAOA   0.0  ,  cplex:-2.0
test_5


constraint A0_leb0 is infeasible due to substitution
constraint A0_leb1 is infeasible due to substitution


success

 optimal value QAOA   1.0  ,  cplex:-1.0
test_6


BasicAerError: 'Number of qubits 28 is greater than maximum (24) for "qasm_simulator".'

In [7]:
qubo = QuadraticProgramToQubo().convert(qps['test_4'])
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver())

In [8]:
exact_res = exact.solve(qubo)
print(exact_res)

optimal function value: -2.0
optimal value: [0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.]
status: SUCCESS


In [9]:
res = qaoa.solve(qubo)
print(res)

optimal function value: 5.0
optimal value: [1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1.]
status: SUCCESS


In [ ]:
exact_res = exact.solve(qps['test_5'])
print(exact_res)
# this kills the Python Kernel...
# Memory Error ?